# **Playing with Cirq**

Check if ```cirq``` is installed

In [1]:
!python -c 'import cirq_google; print(cirq_google.Sycamore)'

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                    (1, 4)───(1, 5)───(1, 6)───(1, 7)
                                    │        │        │        │
                                    │        │        │        │
                           (2, 3)───(2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                           │        │        │        │        │        │
                           │        │        │        │        │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │        │        │        │        │
                  │        │        │        │        │        │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)───(4, 6)───(4, 7)───(4, 8)───(4, 9)
         │        │        │        │        │        │   

It works!

### **Hello Qubit**

In [2]:
import cirq

# pick a qubit
qubit = cirq.GridQubit(0, 0)

# create a circuit that applies a square root of NOT gate & measures the qubit
circuit = cirq.Circuit(cirq.X(qubit) ** 0.5, cirq.measure(qubit, key='m'))
print("Circuit: ")
print(circuit)

# simulate the circuit several times
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results: ")
print(result)

Circuit: 
(0, 0): ───X^0.5───M('m')───
Results: 
m=11011010100101010101


### **Qubits**

To create a quantum circuit, first we must define a set of qubits/quantum registers to act on. 

We can define qubits by 3 ways:
- ```cirq.NamedQubit```: by an abstract name
- ```cirq.LineQubit```: by number in a linear array
- ```cirq.GridQubit```: by 2 numbers in a rectangular lattice

In [3]:
# named qubits for abstract algorithms & algorithms unmapped onto hardware
q0 = cirq.NamedQubit('source')
q1 = cirq.NamedQubit('target')

# create line qubits individually
q3 = cirq.LineQubit(3)

# create line qubits in a range like LineQubit(0), LineQubit(1), LineQubit(2)
q0, q1, q2 = cirq.LineQubit.range(3)

# create grid qubits individually
q4_5 = cirq.GridQubit(4, 5)

# create grid qubits in bulk in a square (16) from (0,0) to (3,3)
qubits = cirq.GridQubit.square(4)

### **Gates & Operations**

Now we will create operations using qubits to be used in the circuit. 
- ```Gate``` is an effect that can be applied to a set of qubits.
- ```Operation``` is a gate applied to a set of qubits.

```cirq.H``` is a quantum Hadamard (fundamental quantum gate) which is a ```Gate``` object. ```cirq.H(cirq.LineQubit(1))``` is an ```Operation``` object which is the Hadamard gate applied to a specific qubit (line qubit number 1).

In [6]:
# example gates
cnot_gate = cirq.CNOT
pauli_z = cirq.Z

# use exponentiation to get square root gates
sqrt_x_gate = cirq.X**0.5

# some gates can also take parameters
sqrt_sqrt_y = cirq.YPowGate(exponent=0.25)

# create two qubits at once, in a line.
q0, q1 = cirq.LineQubit.range(2)

# example operations
z_op = cirq.Z(q0)
not_op = cirq.CNOT(q0, q1)
sqrt_iswap_op = cirq.SQRT_ISWAP(q0, q1)

# use the gates you specified earlier.
cnot_op = cnot_gate(q0, q1)
pauli_z_op = pauli_z(q0)
sqrt_x_op = sqrt_x_gate(q0)
sqrt_sqrt_y_op = sqrt_sqrt_y(q0)

### **Cirqcuits & Moments**

Now we are ready to construct a quantum circuit. A ```Circuit``` is a collection of ```Moments``` (collection of ```Operations``` that act during same abstract time slice). Each ```Operation``` must be applied to a disjoint set of qubits compared to each of the other ```Operations``` in the ```Moment``` (which can be thought of as a vertical slice of a quantum circuit diagram).

Out of the several different ways circuits can be constructed, Cirq by default attempts to slide operation into earliest possible ```Moment``` when inserted. 

Append each operation one-by-one:

In [7]:
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(3)
circuit.append(cirq.H(qubits[0]))
circuit.append(cirq.H(qubits[1]))
circuit.append(cirq.H(qubits[2]))
print(circuit)

0: ───H───

1: ───H───

2: ───H───


Append some iterable of operations using a preconstructed list:

In [8]:
circuit = cirq.Circuit()
ops = [cirq.H(q) for q in cirq.LineQubit.range(3)]
circuit.append(ops)
print(circuit)

0: ───H───

1: ───H───

2: ───H───


A generator that yields operation can also be used.

In [9]:
# append with generator
circuit = cirq.Circuit()
circuit.append(cirq.H(q) for q in cirq.LineQubit.range(3))
print(circuit)

# initialize with generator
print(cirq.Circuit(cirq.H(q) for q in cirq.LineQubit.range(3)))

0: ───H───

1: ───H───

2: ───H───
0: ───H───

1: ───H───

2: ───H───


All of the Hadamard gates are pushed as far left as possible, and put into the same Moment since none overlap.

Operations applied to same qubits will be put in sequential, insertion-ordered moments. 

As shown below, the two-qubit gates overlap & are placed in consecutive moments

In [10]:
print(cirq.Circuit(cirq.SWAP(q, q + 1) for q in cirq.LineQubit.range(3)))

0: ───×───────────
      │
1: ───×───×───────
          │
2: ───────×───×───
              │
3: ───────────×───


To construct a circuit without Cirq automatically shifting operations all the way to the left, create the circuit moment-by-moment or use a different ```InsertStrategy```.

In [11]:
# creates each gate in a separate moment by passing an iterable of Moments instead of Operations.
print(cirq.Circuit(cirq.Moment([cirq.H(q)]) for q in cirq.LineQubit.range(3)))

0: ───H───────────

1: ───────H───────

2: ───────────H───


### **Circuits & Devices**

**TO DO**